#### This assignment may be worked individually or in pairs. 
#### Enter your name(s) here:

In [1]:
#name(s) here
# Matthew Jagen

# Assignment 3: KNN and NB classifiers

In this assignment you'll implement the K-Nearest Neighbors (KNN) classifier to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the same Diabetic Retinopathy data set which was used in the previous assignment on decision trees.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import time

Q0. Read the data from a CSV file and store it in a Pandas dataframe.

In [3]:
def get_data(filename):
    data = []
    #your code goes here
    data = pd.read_csv(filename, names=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19"])
    return data

# print(get_data("messidor_features.txt"))

You may use the following function to print a confusion matrix:

In [4]:
def print_confusion_matrix(TP, FN, FP, TN):
    table_data = [[TP,FN],[FP,TN]]
    df = pd.DataFrame(table_data, columns =['Predicted 1','Predicted 0'])
    df = df.rename(index={0: 'Actual 1', 1: 'Actual 0'})
    display(df)


## K Nearest Neighbor (KNN) Classifier

Q1. Normalize the data so that each feature value lies between `[0-1]`.

In class, we talked about why scaling the data is critical to KNN. We also talked about how data scaling should be done *inside the cross validataion loop*. This means that the scaling parameters should be based on the **training set only**, in order to prevent data leakage. Then the test data will need to be scaled, using the parameters found on the **training** data.

Fill in the function to take in a training dataset and a test dataset and normalize them correctly. Return the normalized datasets.

Caution: Return NEW datasets that have been normalized - do not normalize the datasets in-place, so that this can be run numerous times without altering the original data or normalizing already normalized data.

Hint: When using dataframes, you can do this without a loop!

In [5]:
def normalize_data(train, test):
    # your code goes here
    train_norm = train.copy()
    test_norm = test.copy()
    
    cols = train_norm.columns
    for col in cols:
# find the values needed for scaling this column using only training data
        min_val = train_norm[col][train_norm[col].idxmin]
        max_val = train_norm[col][train_norm[col].idxmax]
        val_range = max_val-min_val
        
# normalize this column in training data using training data scalar values
        train_norm[col] = train_norm[col] - min_val
        train_norm[col] = train_norm[col] / val_range
# scale this column in test data using training data scalar values
        test_norm[col] = test_norm[col] - min_val
        test_norm[col] = test_norm[col] / val_range
    
    return train_norm, test_norm

# data = get_data("messidor_features.txt")
# scaled_data, scaled_data2 = normalize_data(data, data)
# print(scaled_data2.head(10))
# print(scaled_data.head(10))
    

Q2. The distance calculation method is central to the KNN algorithm. In this assignment you'll be using Euclidean distance. 

Implement a function that takes in one data point (as a list), and the training data (as a dataframe), and calculates the Euclidian distance from the single data point to each of the data points in the training data.

You may return these distances however you want (or may add them to the dataframe).

Hint: 
For KNN, the distance calculations are the most time-consuming part of the algorithm. Even though computing Euclidian distance seems like a simple, and therefore quick, calculation, running it thousands of times, inside of a nested 5-fold cross-validation for example, can cause this algorithm to take a very long time to run, depending on your implementation. 

Remember, you almost never need to loop a Dataframe! Pandas DataFrames have been specifically optimized for fast operations on large datasets, by [vectorizing](https://www.quantifisolutions.com/vectorization-part-2-why-and-what) calculations across all rows at once.

If you use a DataFrame, you should not write a loop to calculate each of the Euclidian distances one at a time. Look at [this post](https://stackoverflow.com/questions/46908388/find-euclidean-distance-from-a-point-to-rows-in-pandas-dataframe?rq=1) for more info.

Caution: Be careful not to use the label in your distance calculation.

In [6]:
def get_distances(point, df):
    # your code goes here
#     adapted from the stackoverflow post linked in Q2
    distances = df[df.columns[0:-1]].sub(np.array(point[0:-1])).pow(2).sum(1).pow(0.5)
    return distances

# data = get_data("messidor_features.txt")
# print(get_distances(data.iloc[0], data))

Q3. Build your KNN classifier.

This function takes in a training set (as a dataframe), a test set (as a dataframe), and a k to use, and classifies all data points in the test set, using the data in the training set and the given k.

It should return the predicted labels for the test set as a list.

Caution: Remember to normalize your data before doing distance calculations.

In [7]:
def run_knn(train_set, test_set, k):
    # your code goes here
#   normalize datasets with training data
    train_norm, test_norm = normalize_data(train_set, test_set)
    
    predictions = []
#   loop to predict points in training data
    for i in test_norm.index:
        point = test_norm.iloc[i].tolist()
        
#       find test point's K nearest neighbors in the training data
        distances = get_distances(point, train_norm)
        distances.sort_values(inplace=True)
        near_neighbors = distances[0:k]
        
#       use the neighbors to predict the class label
        yes_count = 0
        for i in near_neighbors.index:
            yes_count += train_norm["19"][i]
        prediction = 0
        if yes_count/k >= 0.5: #if for some reason K is even, bias 50/50 to positive
            prediction = 1
            
        predictions.append(prediction)
    return predictions

# data = get_data("messidor_features.txt")
# print(run_knn(data.copy(), data.copy(), 9))

Q4. Find the best value of k for this data. 

Try k ranging from 1 to 10 (odds only). For each k value, use a 5-fold cross validation to evaluate the accuracy with that k. In each fold of CV, divide your data into a training set and a validation set. Print out the best value of k and the accuracy achieved with that value. Return the best value of k. If there is a tie for best k, use the lowest of the k values.

Hint: This is the *inner* loop of a nested cross validation.

In [8]:
def find_best_k(data):
    # your code goes here   
    num_folds = 5 
    k_values = [1, 3, 5, 7, 9]
    k_accuracies = []
    for k in k_values:
        accuracies = []
        for i in range(num_folds):
#           split data into test set (from partition start to partition end) and training set (everything else)
            partition_start_idx = int((i)*(len(data)/num_folds))
            partition_end_idx = int((i+1)*(len(data)/num_folds))
            test_set = data.iloc[partition_start_idx:partition_end_idx]
            train_set_first_half = data.iloc[:partition_start_idx]
            train_set_second_half = data.iloc[partition_end_idx:]
            train_set = pd.concat([train_set_first_half, train_set_second_half])
            test_set.reset_index(inplace=True)
            train_set.reset_index(inplace=True)
            
#           use run_knn to get predictions on the test set from the training set and then calculate the accuracy of this fold
            predictions = run_knn(train_set, test_set, k)
            num_correct = 0
            for i in test_set.index:
                if test_set["19"][i] == predictions[i]:
                    num_correct += 1
            accuracies.append(num_correct/len(predictions))
            
#       add the average accuracy for this k value to the k_accuracies list
        total = 0.0
        for accuracy in accuracies:
            total += accuracy
        k_accuracies.append(total/len(accuracies))
        
#   find the k value with the highest average accuracy
    max_acc_index = 0
    for i in range(len(k_accuracies)):
        if k_accuracies[i] > k_accuracies[max_acc_index]:
            max_acc_index = i
    best_k = k_values[max_acc_index]
    return best_k

# data = get_data("messidor_features.txt")
# print(find_best_k(data))

Q5. Now measure the accuracy of your classifier using 5-fold cross validation. 

In each fold of this CV, divide your data into a training set and a test set. The training set should get sent through your code for Q4, resulting in a value of k to use. Using that k, calculate an accuracy on the test set. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. 

Print the accuracy, the confusion matrix, and the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [9]:
# read in data
data = get_data('messidor_features.txt')
start_time = time.time()

# your code goes here
num_folds = 5
accuracies = []
TP = 0
FP = 0
TN = 0
FN = 0
for i in range(num_folds):
#   split data into test set (from partition start to partition end) and training set (everything else)
    partition_start_idx = int((i)*(len(data)/num_folds))
    partition_end_idx = int((i+1)*(len(data)/num_folds))
    test_set = data.iloc[partition_start_idx:partition_end_idx]
    train_set_first_half = data.iloc[:partition_start_idx]
    train_set_second_half = data.iloc[partition_end_idx:]
    train_set = pd.concat([train_set_first_half, train_set_second_half])
    test_set.reset_index(inplace=True)
    train_set.reset_index(inplace=True)
    
#   find best k and use it to get the accuracy for this fold (also increment counts for our confusion matrix)
    best_k = find_best_k(train_set)
    predictions = run_knn(train_set, test_set, best_k)
    num_correct = 0
    j = 0
    for label in test_set["19"]:
        if label == predictions[j]:
            num_correct += 1
            if predictions[j] == 1:
                TP += 1
            else:
                TN += 1
        else:
            if predictions[j] == 1:
                FP += 1
            else:
                FN += 1
        j += 1
    
    accuracies.append(num_correct/len(predictions))

# calculate the final accuracy measurement
total = 0.0
for accuracy in accuracies:
    total += accuracy
accuracy = total/len(accuracies)

# print out the accuracy, precision, recall, and confusion matrix
print("Accuracy: ", str(accuracy))
precision = TP/(TP+FP)
print("Precision (+): ", str(precision))
recall = TP/(TP+FN)
print("Recall (+): ", str(recall))
print_confusion_matrix(TP, FN, FP, TN)
    
end_time = time.time()
print('\nTotal time:', end_time - start_time)

Accuracy:  0.6156521739130435
Precision (+):  0.6556169429097606
Recall (+):  0.5826513911620295


,Predicted 1,Predicted 0
Actual 1,356,255
Actual 0,187,352



Total time: 37.9044885635376
